# Spacex Data Collection using Webscraping

In [19]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import pprint

In [20]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [21]:
#URL to be scraped
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [22]:
#fetch the data from the URL
response = requests.get(static_url)

In [23]:
#creating a BeautifulSoup object from the data
soup = BeautifulSoup(response.content)

In [24]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [25]:
#listing all the table tags
html_tables = soup.find_all('table')


In [26]:
for i in range(len(html_tables)):
    print(html_tables[i].find('th'))

None
None
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight No.
</th>
<th scope="col">Flight
<p>No.
</p>
</th>
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th class="navbox-title" colspan="2" scope="col"><link href="mw-data:TemplateStyles:r1129693374" rel="mw-dedu

In [27]:
pprint.pprint(html_tables[2])

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [28]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]

In [29]:
#identifying the actual headings
headings = first_launch_table.find_all('th')

for i in range(len(headings)):
    print(len(headings[i].text))

11
19
20
12
11
13
6
9
14
15
2
2
2
2
2
2
2


In [30]:
#create a list of column names by scraping headings from the table
column_names = []

for h in headings:
    #some missing headings had a length <=2
    if len(h.text)>2:
        column_names.append(h.text.strip())

print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


In [31]:
launch_dict= dict.fromkeys(column_names)


# Remove an irrelvant column
del launch_dict['Date andtime (UTC)']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload[c]'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launchoutcome'] = []
# Added some new columns
launch_dict['Version,Booster [b]']=[]
launch_dict['Boosterlanding']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [32]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
                    
            time = datatimelist[1]
            launch_dict['Time'].append(time) 
            print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version,Booster [b]'].append(bv) 
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload[c]'].append(payload) 
            print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass) 
            print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit) 
            print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].text.strip()
            launch_dict['Customer'].append(customer) 
            print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launchoutcome'].append(launch_outcome) 
            print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Boosterlanding'].append(booster_landing) 
            print(booster_landing)
            
            

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
Dragon Spacecraft Qualification Unit
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
Dragon
LEO
NASA (COTS)
NRO
Success
Failure
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
Dragon
LEO
NASA (COTS)
Success
No attempt

4
8 October 2012
00:35
F9 v1.0B0006.1
CCAFS
SpaceX CRS-1
SpaceX CRS-1
LEO
NASA (CRS)
Success

No attempt
5
1 March 2013
15:10
F9 v1.0B0007.1
CCAFS
SpaceX CRS-2
SpaceX CRS-2
LEO
NASA (CRS)
Success

No attempt

6
29 September 2013
16:00
F9 v1.1B1003
VAFB
CASSIOPE
CASSIOPE
Polar orbit
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
SES-8
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
Thaicom 6
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
SpaceX CRS-3
LEO
NASA (CRS)
Success

Controlled
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
Orbcomm-OG2
LEO
Orbcomm
Success
Con

KSC
Starlink
Starlink
LEO
SpaceX
Success

Failure
84
22 April 2020
19:30
F9 B5
KSC
Starlink
Starlink
LEO
SpaceX
Success

Success
85
30 May 2020
19:22
F9 B5
KSC
Crew Dragon Demo-2
Crew Dragon Demo-2
LEO
NASA (CCDev)
Success

Success
86
4 June 2020
01:25
F9 B5
CCAFS
Starlink
Starlink
LEO
SpaceX
Success

Success
87
13 June 2020
09:21
F9 B5
CCAFS
Starlink
Starlink
LEO
SpaceXPlanet Labs
Success

Success
88
30 June 2020
20:10:46
F9 B5
CCAFS
GPS III
GPS III
MEO
U.S. Space Force[530]
Success

Success
89
20 July 2020
21:30
F9 B5B1058.2
CCAFS
ANASIS-II
ANASIS-II
GTO
Republic of Korea Army
Success

Success
90
7 August 2020
05:12
F9 B5
KSC
Starlink
Starlink
LEO
SpaceXSpaceflight Industries (BlackSky)
Success

Success
91
18 August 2020
14:31
F9 B5B1049.6
CCAFS
Starlink
Starlink
LEO
SpaceXPlanet Labs
Success

Success
92
30 August 2020
23:18
F9 B5
CCAFS
SAOCOM 1B
SAOCOM 1B
SSO
CONAEPlanetIQTyvak
Success

Success
93
3 September 2020
12:46:14
F9 B5B1060.2
KSC
Starlink
Starlink
LEO
SpaceX
Success

Succe

In [33]:
launch_dict

{'Flight No.': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '109',
  '110',
  '111',
  '112',
  '113',
  '114',
  '115',
  '116',
  '117',
  '118',
  '119',
  '120',
  '121'],
 'Ve

In [34]:
for column, values in launch_dict.items():
    print(f"lenght of {column}: {len(values)}")

lenght of Flight No.: 121
lenght of Version,Booster [b]: 121
lenght of Launch site: 121
lenght of Payload[c]: 121
lenght of Payload mass: 121
lenght of Orbit: 121
lenght of Customer: 121
lenght of Launchoutcome: 121
lenght of Boosterlanding: 121
lenght of Date: 121
lenght of Time: 121


In [35]:
df = pd.DataFrame(launch_dict)

In [36]:
df.head()

,Flight No.,"Version,Booster [b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,Date,Time
0,1,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure,4 June 2010,18:45
1,2,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA (COTS)\nNRO,Success,Failure,8 December 2010,15:43
2,3,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA (COTS),Success,No attempt\n,22 May 2012,07:44
3,4,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS),Success\n,No attempt,8 October 2012,00:35
4,5,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS),Success\n,No attempt\n,1 March 2013,15:10


In [37]:
df.rename(columns={'Version,Booster [b]': 'Version Booster', 'Payload[c]':'Payload', 'Launchoutcome': 'Launch outcome', 'Boosterlanding':'Booster landing'}, inplace=True)

In [38]:
df.head()

,Flight No.,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing,Date,Time
0,1,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure,4 June 2010,18:45
1,2,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA (COTS)\nNRO,Success,Failure,8 December 2010,15:43
2,3,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA (COTS),Success,No attempt\n,22 May 2012,07:44
3,4,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA (CRS),Success\n,No attempt,8 October 2012,00:35
4,5,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA (CRS),Success\n,No attempt\n,1 March 2013,15:10


In [39]:
df.to_csv('spacex_web_scraped.csv', index=False)